In [54]:
from delta import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import timeit

In [3]:
builder = (SparkSession.builder
           .appName("optimize-table-partitions-delta")
           .master("spark://spark-master:7077")
           .config("spark.executor.memory", "512m")
           .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
           .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog"))

spark = configure_spark_with_delta_pip(builder).getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

:: loading settings :: url = jar:file:/usr/local/lib/python3.10/dist-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a4ea6b6e-1365-4798-a1ed-32d6667a2ddd;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 310ms :: artifacts dl 22ms
	:: modules in use:
	io.delta#delta-core_2.12;2.4.0 from central in [default]
	io.delta#delta-storage;2.4.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0   |   0 

In [12]:
%load_ext sparksql_magic
%config SparkSql.limit=20

In [45]:
# Create some sample data frames
# A large data frame with 1 million rows
large_df = (spark.range(0, 1000000)
            .withColumn("salary", 100*(rand() * 100).cast("int"))
            .withColumn("gender", when((rand() * 2).cast("int") == 0, "M").otherwise("F"))
            .withColumn("country_code", 
                        when((rand() * 4).cast("int") == 0, "US")
                        .when((rand() * 4).cast("int") == 1, "CN")
                        .when((rand() * 4).cast("int") == 2, "IN")
                        .when((rand() * 4).cast("int") == 3, "BR")
                        .otherwise('RU')))
large_df.show(5)

+---+------+------+------------+
| id|salary|gender|country_code|
+---+------+------+------------+
|  0|  3100|     M|          US|
|  1|  4300|     M|          CN|
|  2|  3000|     M|          IN|
|  3|  4500|     F|          US|
|  4|  5900|     F|          RU|
+---+------+------+------------+
only showing top 5 rows



In [47]:
(large_df.write
 .format("delta")
 .mode("overwrite")
 .save("../data/tmp/large_delta"))

In [48]:
(large_df.write
 .format("delta")
 .mode("overwrite")
 .partitionBy("country_code")
 .option("overwriteSchema", "true")
 .save("../data/tmp/large_delta_partitioned"))

In [55]:
non_partitioned_query = "spark.sql(\"SELECT country_code,gender, COUNT(*) AS employees FROM delta.`/opt/workspace/data/tmp/large_delta` GROUP BY ALL ORDER BY employees DESC\").show()"
non_partitioned_time = timeit.timeit(non_partitioned_query, number=1, globals=globals())
print(f"Non-partitioned query time: {non_partitioned_time} seconds")

+------------+------+---------+
|country_code|gender|employees|
+------------+------+---------+
|          RU|     F|   158374|
|          RU|     M|   158085|
|          US|     M|   124924|
|          US|     F|   124921|
|          CN|     F|    94448|
|          CN|     M|    93464|
|          IN|     F|    70416|
|          IN|     M|    70189|
|          BR|     F|    52726|
|          BR|     M|    52453|
+------------+------+---------+

Non-partitioned query time: 1.8541588850002881 seconds


In [56]:
partitioned_query = "spark.sql(\"SELECT country_code,gender, COUNT(*) AS employees FROM delta.`/opt/workspace/data/tmp/large_delta_partitioned` GROUP BY ALL ORDER BY employees DESC\").show()"
partitioned_time = timeit.timeit(partitioned_query, number=1, globals=globals())
print(f"Partitioned query time: {partitioned_time} seconds")


+------------+------+---------+
|country_code|gender|employees|
+------------+------+---------+
|          RU|     F|   158374|
|          RU|     M|   158085|
|          US|     M|   124924|
|          US|     F|   124921|
|          CN|     F|    94448|
|          CN|     M|    93464|
|          IN|     F|    70416|
|          IN|     M|    70189|
|          BR|     F|    52726|
|          BR|     M|    52453|
+------------+------+---------+

Partitioned query time: 0.9437477390001732 seconds


In [57]:
spark.stop()